In [1]:
from readDB import *
from preProcessing import preProcessing
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
import operator


def fn_nvarFilt(poses, nvar):
    nvjr = list(nvar)
    ja = lambda tag: tag if tag != 'j' else 'a'
    nvars = [(p0, ja(p1[:1].lower())) for p0, p1 in poses if p1[:1] in nvjr]
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(nvar[0], pos=nvar[1]) for nvar in nvars]

def FindN(query):
    tokens = word_tokenize(query.lower())
    poses = pos_tag(tokens)
    nltk_res = fn_nvarFilt(poses, 'N')
    nltk_freq = defaultdict(int)
    for token in nltk_res:
        nltk_freq[token] += 1
    return nltk_res

# FindN 을 사용하여 쿼리에서 명사를 추출하여 Boolean에 사용

q=input() #쿼리 입력 ex) warm chicken with onion

candidate=FindN(q)   # candidate = [chicken,onion]
print(candidate)


test
54
107
90
117
73
68
74
73
94
70
75
61
82
116
71
76
106
104
51
92
82
71
57
99
89
79
71
88
77
87
75
81
74
89
138
117
84
83
79
48
98
72
85
75
79
103
80
100
77
78
74
63
67
75
56
81
75
85
87
68
57
48
50
92
85
106
108
97
71
72
87
108
105
77
98
80
84
81
58
66
79
72
98
71
74
66
89
68
86
93
71
73
87
67
76
53
77
84
75
102
{'Turkey Burritos': {'specif': 0.007966459283539817, 'turkey': 0.04001200449989038, 'shred': 0.009134561817783193, 'de': 0.007966459283539817, 'boil': 0.009134561817783193, 'avocado': 0.008505583530113683, 'gram': 0.03186583713415927, 'pile': 0.007966459283539817, 'enclos': 0.007966459283539817, 'well': 0.008685291612304972, 'jenni': 0.007966459283539817, 'measur': 0.007996410630571698, 'divid': 0.00883504834746438, 'minut': 0.01635283645246661, 'flour': 0.008415729489018038, 'chili': 0.0169512643661636, 'done': 0.008056313324635462, 'bring': 0.008924902388560023, 'cheddar': 0.008295924100890514, 'cook': 0.02180338258532839, 'mixtur': 0.01793965881821569, 'fold': 0.0083558

warm chicken with onion
['chicken', 'onion']


In [10]:
def boolean_Ingredient_all(q_ing, db_ing) :
    ranked = []
    lg =[]
    for food in db.keys():
        flag = 0
        s = db[food]['ingredient']
        for qu in (q_ing):
            if qu not in s:
                break;
            else:
                flag += 1 
        if flag == len(q_ing):
            querylen =len(q_ing)
            inglen=len(s.split(' '))
            rank_value = querylen/inglen
            lg.append([food, rank_value])
    
    lg =sorted(lg, key=operator.itemgetter(1))
    return lg

def boolean_Ingredient_nameonly(q_ing, db_ing) :
    ranked = []
    lg =[]
    for food in db.keys():
        flag = 0
        s = db[food]['ingredient']
        for qu in (q_ing):
            if qu not in s:
                break;
            else:
                flag += 1 
        if flag == len(q_ing):
            querylen =len(q_ing)
            inglen=len(s.split(' '))
            rank_value = querylen/inglen
            lg.append(food)
    return lg

def boolean_Ingredient(q_ing, db_ing) :
    ranked = []
    lg =[]
    for food in db.keys():
        flag = 0
        s = db[food]['ingredient']
        for qu in (q_ing):
            if qu not in s:
                break;
            else:
                flag += 1 
        if flag == len(q_ing):
            querylen =len(q_ing)
            inglen=len(s.split(' '))
            rank_value = querylen/inglen
            lg.append([food, rank_value])
    
    lg =sorted(lg, key=operator.itemgetter(1))
    for i in range(5):
        if (len(lg)-i-1) == -1:
            break;
        
        ranked.append(lg[len(lg)-1-i][0])
        
    return ranked

def boolean_Ingredient_weight(q_ing, db_ing) :
    ranked_with_weight = []
    lg =[]
    for food in db.keys():
        flag = 0
        s = db[food]['ingredient']
        for qu in (q_ing):
            if qu not in s:
                break;
            else:
                flag += 1 
        if flag == len(q_ing):
            querylen =len(q_ing)
            inglen=len(s.split(' '))
            rank_value = querylen/inglen
            lg.append([food, rank_value])
    
    lg =sorted(lg, key=operator.itemgetter(1))
    for i in range(5):
        if (len(lg)-i-1) == -1:
            break;
        ranked_with_weight.append(lg[len(lg)-i-1])
        
    return ranked_with_weight
data_name = boolean_Ingredient_nameonly(candidate, db) # 소팅되지 않은 해당 음식들 전부
data_all = boolean_Ingredient_all(candidate, db) # 소팅되지 않은 음식들의 값 전부 계산
data = boolean_Ingredient(candidate, db) # 소팅된 음식들의 weight 빼고
data_weight = boolean_Ingredient_weight(candidate, db) # data에는 chicken과 onion이 포함된 커리만 출력됨 - ranking을 통해 5위까지 출력
print(data_name)
print(data_all)
print(data)
print(data_weight)

['Sema Chicken', 'Garlic Ginger Chicken Potstickers', 'Sesame Chicken Potstickers', 'Asian-Style Cobb Salad', 'Coconut Curry Chicken Soup', 'Asian Style Chicken Nuggets With Lemon Glaze', 'Sweet And Sour Stir-Fried Chicken', 'Chinese Orange Chicken', 'Chinese Chicken Salad', 'Chinese Beef with Broccoli', 'Slow Cooker Indian Butter Chicken', 'Easy Chicken Tikka Masala', 'Instant Pot Mushroom Risotto', 'Italian Wedding Soup', 'Mushroom Ramen', 'Asian Style Nachos', 'Korean Chicken Kabobs', 'Korean Curry', 'Instant Pot Chicken Tortilla Soup', 'Roasted Shrimp Enchiladas with Jalapeno Cream Sauce', 'Chicken Khao Soi', 'Thai Red Curry Noodle Soup', 'Thai Green Curry Chicken Soup', 'Thai Sweet Potato and Carrot Soup', 'Thai Peanut Chicken Noodles', 'Slow Cooker Corn Chowder']
[['Italian Wedding Soup', 0.021505376344086023], ['Roasted Shrimp Enchiladas with Jalapeno Cream Sauce', 0.023809523809523808], ['Chicken Khao Soi', 0.02702702702702703], ['Thai Red Curry Noodle Soup', 0.0277777777777777